In [6]:
import datetime
import os
import meteostat
import numpy as np
import pandas as pd
import psycopg2 as pg
from psycopg2.extras import execute_values
import statsapi
from tqdm import tqdm

In [60]:
stadiums = pd.read_csv("stadiums.csv")
stadiums.head()

,venue,team,abbrev,lat,long
0,Chase Field,Arizona Diamondbacks,AZ,33.445278,-112.066944
1,Turner Field,Atlanta Braves,ATL,33.735278,-84.389444
2,Oriole Park at Camden Yards,Baltimore Orioles,BAL,39.283889,-76.621667
3,Fenway Park,Boston Red Sox,BOS,42.346389,-71.097500
4,Wrigley Field,Chicago Cubs,CHC,41.948333,-87.655556


In [7]:
conn = pg.connect(
    dbname="mlb",
    user="postgres",
    password=os.environ.get("MLB_DB_PW"),
    host="database-1.cqhpcblctccg.us-east-1.rds.amazonaws.com",
)
cur = conn.cursor()

In [51]:
query = "SELECT team_id, abbrev FROM team;"
cur.execute(query)
teams = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])
teams.head()

,team_id,abbrev
0,7,OAK
1,8,LAD
2,9,SD
3,25,SF
4,27,SEA


In [58]:
assert set(teams["abbrev"]) == set(stadiums["Abbreviation"])

In [63]:
df = teams.merge(stadiums)[["team_id", "lat", "long"]].set_index("team_id")
df.head()

,lat,long
team_id,,
7,37.751667,-122.200556
8,34.073611,-118.240000
9,32.707300,-117.156600
25,37.778333,-122.389444
27,47.591389,-122.332500


In [114]:
stations = meteostat.Stations()
station_ids = {}

for team_id in df.index:
    team_stations = stations.nearby(df["lat"][team_id], df["long"][team_id]).fetch(10)

    for station_idx in team_stations.index:
        if team_stations["hourly_start"][station_idx] < datetime(
            2008, 1, 1
        ) and team_stations["hourly_end"][station_idx] > datetime(2024, 1, 1):
            station_ids[team_id] = station_idx
            break

assert set(teams["team_id"]) == station_ids.keys()

In [119]:
df["nearest_station_id"] = df.index.map(station_ids)
df.head()

,lat,long,nearest_station_id
team_id,,,
7,37.751667,-122.200556,72493
8,34.073611,-118.240000,KCQT0
9,32.707300,-117.156600,72290
25,37.778333,-122.389444,72493
27,47.591389,-122.332500,KBFI0


In [8]:
query = "SELECT game_id, game_date, home_team_id FROM game;"
cur.execute(query)
game_info = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])
game_info.head()

,game_id,game_date,home_team_id
0,257754,2009-04-03,16
1,257751,2009-04-03,8
2,706921,2022-04-05,8
3,332491,2012-03-29,9
4,332498,2012-03-29,19


In [130]:
game_data = game_info.merge(df, left_on="home_team_id", right_on=df.index).set_index(
    "game_id"
)
game_data.head()

,game_date,home_team_id,lat,long,nearest_station_id
game_id,,,,,
257754,2009-04-03,16,39.905833,-75.166389,72408
257751,2009-04-03,8,34.073611,-118.240000,KCQT0
706921,2022-04-05,8,34.073611,-118.240000,KCQT0
332491,2012-03-29,9,32.707300,-117.156600,72290
332498,2012-03-29,19,32.751389,-97.082778,KGPM0


In [175]:
# get weather for each game
values = []

for game_id in game_data.index:
    start = datetime.datetime.strptime(
        f"{game_data['game_date'][game_id].strftime('%Y-%m-%d')} 00:00",
        "%Y-%m-%d %H:%M",
    )
    end = start + datetime.timedelta(hours=12)
    data = meteostat.Hourly(
        game_data["nearest_station_id"][game_id], start, end
    ).fetch()
    values.append((game_id,) + tuple(data.iloc[0].values))

In [176]:
# create DataFrame of weather for each game
data_cols = [
    "game_id",
    "temp",
    "dwpt",
    "rhum",
    "prcp",
    "snow",
    "wdir",
    "wspd",
    "wpgt",
    "pres",
    "tsun",
    "coco",
]
game_weather = pd.DataFrame(values, columns=data_cols)
game_weather.head()

,game_id,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,257754,15.6,12.2,80.0,0.0,NaN,130.0,9.4,NaN,1012.3,NaN,NaN


In [9]:
for game_id in tqdm(game_info["game_id"].unique(), position=0, leave=True):
    data = statsapi.get("game", params={"gamePk": game_id}, force=False)
    print(data["gameData"]["weather"])

  0%|          | 0/44797 [00:00<?, ?it/s]

  0%|          | 1/44797 [00:00<10:39:44,  1.17it/s]

{'condition': 'Cloudy', 'temp': '66', 'wind': '17 mph, L To R'}


  0%|          | 2/44797 [00:01<10:38:25,  1.17it/s]

{'condition': 'Clear', 'temp': '59', 'wind': '6 mph, Out To CF'}


KeyboardInterrupt: 

In [11]:
data["gameData"]["venue"]

{'id': 22,
 'name': 'Dodger Stadium',
 'link': '/api/v1/venues/22',
 'location': {'address1': '1000 Vin Scully Avenue',
  'city': 'Los Angeles',
  'state': 'California',
  'stateAbbrev': 'CA',
  'postalCode': '90012-1199',
  'defaultCoordinates': {'latitude': 34.07368, 'longitude': -118.24053},
  'azimuthAngle': 26.0,
  'elevation': 515,
  'country': 'USA',
  'phone': '(323) 224-1500'},
 'timeZone': {'id': 'America/Los_Angeles',
  'offset': -8,
  'offsetAtGameTime': -7,
  'tz': 'PST'},
 'fieldInfo': {'capacity': 56000,
  'turfType': 'Grass',
  'roofType': 'Open',
  'leftLine': 330,
  'leftCenter': 385,
  'center': 395,
  'rightCenter': 385,
  'rightLine': 330},
 'active': True,
 'season': '2009'}

In [14]:
data["gameData"]["venue"]

{'id': 22,
 'name': 'Dodger Stadium',
 'link': '/api/v1/venues/22',
 'location': {'address1': '1000 Vin Scully Avenue',
  'city': 'Los Angeles',
  'state': 'California',
  'stateAbbrev': 'CA',
  'postalCode': '90012-1199',
  'defaultCoordinates': {'latitude': 34.07368, 'longitude': -118.24053},
  'azimuthAngle': 26.0,
  'elevation': 515,
  'country': 'USA',
  'phone': '(323) 224-1500'},
 'timeZone': {'id': 'America/Los_Angeles',
  'offset': -8,
  'offsetAtGameTime': -7,
  'tz': 'PST'},
 'fieldInfo': {'capacity': 56000,
  'turfType': 'Grass',
  'roofType': 'Open',
  'leftLine': 330,
  'leftCenter': 385,
  'center': 395,
  'rightCenter': 385,
  'rightLine': 330},
 'active': True,
 'season': '2009'}

In [ ]:
found_stadiums = {}

stadium = data["gameData"]["venue"]

mlb_id = stadium["mlb_id"]
season = stadium["season"]

if mlb_id not in found_stadiums:
    found_stadiums[mlb_id] = {"seasons": []}

    name = stadium["name"]
    address = stadium["location"]["address1"]
    city = stadium["location"]["city"]
    state = stadium["location"]["state"]
    state_abbrev = stadium["location"]["stateAbbrev"]
    postal_code = stadium["location"]["postalCode"]
    country = stadium["defaultCoordinates"]["country"]
    latitude = stadium["defaultCoordinates"]["latitude"]
    longitude = stadium["defaultCoordinates"]["longitude"]
    azimuth = stadium["defaultCoordinates"]["azimuthAngle"]
    elevation = stadium["defaultCoordinates"]["elevation"]
    tz_offset = stadium["timeZone"]["offset"]
    tz = stadium["timeZone"]["tz"]
    active = stadium["active"]

    query = """
    INSERT INTO stadium (
        mlb_id,
        name,
        address,
        city,
        state,
        state_abbrev,
        postal_code,
        country,
        latitude,
        longitude,
        azimuth,
        elevation,
        tz_offset,
        tz,
        active
        )
        VALUES %s
        ON CONFLICT (mlb_id) DO NOTHING;
        RETURNING stadium_id;
        """.format(
        mlb_id,
        name,
        address,
        city,
        state,
        state_abbrev,
        postal_code,
        country,
        latitude,
        longitude,
        azimuth,
        elevation,
        tz_offset,
        tz,
        active,
    )
    cur.execute(query)
    stadium_id = cur.fetchall()[0]
    conn.commit()
    found_stadiums[mlb_id]["id"] = stadium_id


if season not in found_stadiums[mlb_id]["seasons"]:
    found_stadiums[mlb_id]["seasons"].append(season)

    capacity = stadium["fieldInfo"]["capacity"]
    turf_type = stadium["fieldInfo"]["turfType"]
    roof_type = stadium["fieldInfo"]["roofType"]
    distance_to_left = stadium["fieldInfo"]["leftLine"]
    distance_to_lc = stadium["fieldInfo"]["leftCenter"]
    distance_to_center = stadium["fieldInfo"]["center"]
    distance_to_rc = stadium["fieldInfo"]["rightCenter"]
    distance_to_right = stadium["fieldInfo"]["rightLine"]

    query = """
    INSERT INTO stadium_season (
        stadium_id,
        season,
        capacity,
        turf_type,
        roof_type,
        distance_to_left,
        distance_to_lc,
        distance_to_center,
        distance_to_rc,
        distance_to_right
        )
        VALUES %s
        ON CONFLICT (stadium_id, season) DO NOTHING;
        """.format(
        found_stadiums[mlb_id]["id"],
        season,
        capacity,
        turf_type,
        roof_type,
        distance_to_left,
        distance_to_lc,
        distance_to_center,
        distance_to_rc,
        distance_to_right,
    )
    cur.execute(query)
    conn.commit()

In [ ]:
# TO DO: Get game times to get accurate hourly weather
# TO DO: Get stadium orientation to get accurate wind